In [9]:
import os
import re
import ast
import sys
import time
import json
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from sklearn.preprocessing import LabelEncoder
from math import sin, cos, sqrt, atan2, radians
from sklearn.preprocessing import OneHotEncoder
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [11]:
def chromeDriver():  
    
    path_to_chromedriver =  '/home/nimori/Desktop/chrome_driver/chromedriver'

    options = webdriver.ChromeOptions()
    
    prefs = {
              "download.default_directory": '../Data/',
              "directory_upgrade": True,
              "extensions_to_open": "",
              "translate_whitelists": {"hi":"en"},
              "translate":{"enabled":"true"}
            }
    
    options.add_experimental_option("prefs", prefs)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-setuid-sandbox")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-logging")
    options.add_argument("--disable-login-animations")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-default-apps")
#     options.add_argument("headless")

    main_browser = webdriver.Chrome(executable_path = path_to_chromedriver, chrome_options = options)
    
    return main_browser

In [63]:
def fps(main_browser):
    
    main_browser.get('http://food.raj.nic.in/DistrictWiseCategoryDetails.aspx')
    
    main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[4]/td/table/tbody/tr/td[1]/input').click()
    
    all_table_xpath= '/html/body/form/table/tbody/tr[5]/td/div/div/table'
    if not os.path.exists('./data/DistrictWiseCategoryDetails/all.csv'):   
        all_df= tableToDf(main_browser, all_table_xpath)
        all_df.to_csv('./data/DistrictWiseCategoryDetails/all.csv', index= False, encoding= 'utf-8')
    len_all= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[5]/td/div/div/table/tbody/tr[34]/td[1]').text
    len_all= len_all.strip().strip('.')
    #print(len_all)
    time.sleep(2)
    
    for i in range(int(len_all)):  #i= 33                 
        rural= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[5]/td/div/div/table/tbody/tr['+str(i+2)+']/td[3]/a')
        rural_text= rural.text                    
        rural.click()        
        rural_table_xpath= '/html/body/form/table/tbody/tr[6]/td/div/div/div/table'
        if not os.path.isdir('./data/DistrictWiseCategoryDetails/rural'):
            os.makedirs('./data/DistrictWiseCategoryDetails/rural')
        if not os.path.isfile('./data/DistrictWiseCategoryDetails/rural/'+rural_text+'.csv'):
            rural_df= tableToDf(main_browser, rural_table_xpath)       
            rural_df.to_csv('./data/DistrictWiseCategoryDetails/rural/'+rural_text+'.csv', index= False, encoding= 'utf-8')
        len_rural= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[6]/td/div/div/div/table/tbody/tr[10]/td[1]').text
        len_rural= len_rural.strip().strip('.')        
        time.sleep(2)
        
        for j in range(int(len_rural)): #9
            block= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[6]/td/div/div/div/table/tbody/tr['+str(j+2)+']/td[2]/a')
            block_text= block.text                       
            block.click()         
            block_table_xpath= '/html/body/form/table/tbody/tr[6]/td/div/div/div/table'
            if not os.path.isdir('./data/DistrictWiseCategoryDetails/rural/block'):
                os.makedirs('./data/DistrictWiseCategoryDetails/rural/block')
            if not os.path.isfile('./data/DistrictWiseCategoryDetails/rural/block/'+block_text+'.csv'):
                block_df= tableToDf(main_browser, block_table_xpath)       
                block_df.to_csv('./data/DistrictWiseCategoryDetails/rural/block/'+block_text+'.csv', index= False, encoding= 'utf-8')
            len_block= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[6]/td/div/div/div/table/tbody/tr[22]/td[1]').text
            len_block= len_block.strip().strip('.')
            time.sleep(2)
            
            for k in range(int(len_block)): #21           
                panch= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[6]/td/div/div/div/table/tbody/tr['+str(k+2)+']/td[2]/a')
                panch_text= panch.text
                panch.click()   
                panch_table_xpath= '/html/body/form/table/tbody/tr[7]/td/div/div/table'
                if not os.path.isdir('./data/DistrictWiseCategoryDetails/rural/block/panch'):
                    os.makedirs('./data/DistrictWiseCategoryDetails/rural/block/panch')
                if not os.path.isfile('./data/DistrictWiseCategoryDetails/rural/block/panch/'+panch_text+'.csv'):
                    panch_df= tableToDf(main_browser, panch_table_xpath)       
                    panch_df.to_csv('./data/DistrictWiseCategoryDetails/rural/block/panch/'+panch_text+'.csv', index= False, encoding= 'utf-8')
                len_panch= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[7]/td/div/div/table/tbody/tr[7]/td[1]').text
                len_panch= len_panch.strip().strip('.')        
                time.sleep(2)
                
                for l in range(int(len_panch)):
                    village= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[7]/td/div/div/table/tbody/tr['+str(l+2)+']/td[2]/a')
                    village_text= village.text
                    village.click()       
                    village_table_xpath= '/html/body/form/table/tbody/tr[8]/td/div/div/table'
                    if not os.path.isdir('./data/DistrictWiseCategoryDetails/rural/block/panch/village'):
                        os.makedirs('./data/DistrictWiseCategoryDetails/rural/block/panch/village')
                    if not os.path.isfile('./data/DistrictWiseCategoryDetails/rural/block/panch/village/'+village_text+'.csv'):
                        village_df= tableToDf(main_browser, village_table_xpath)       
                        village_df.to_csv('./data/DistrictWiseCategoryDetails/rural/block/panch/village/'+village_text+'.csv', index= False, encoding= 'utf-8')
                    len_village= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[8]/td/div/div/table/tbody/tr[3]/td[1]').text
                    len_village= len_village.strip().strip('.')
                    time.sleep(2)
                    
                    for m in range(int(len_village)):
                        fps= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[8]/td/div/div/table/tbody/tr['+str(m+2)+']/td[2]/a')
                        fps_text= fps.text
                        fps.click()       
                        fps_table_xpath= '/html/body/form/table/tbody/tr[9]/td/div/div/table'
                        if not os.path.isdir('./data/DistrictWiseCategoryDetails/rural/block/panch/village/fps'):
                            os.makedirs('./data/DistrictWiseCategoryDetails/rural/block/panch/village/fps')
                        if not os.path.isfile('./data/DistrictWiseCategoryDetails/rural/block/panch/village/fps/'+fps_text+'.csv'):
                            fps_df= tableToDf(main_browser, fps_table_xpath)       
                            fps_df.to_csv('./data/DistrictWiseCategoryDetails/rural/block/panch/village/fps/'+fps_text+'.csv', index= False, encoding= 'utf-8')
                        len_fps= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[9]/td/div/div/table/tbody/tr[101]/td[1]').text
                        len_fps= len_fps.strip().strip('.')
                        time.sleep(2)
                        
                        for n in range(int(len_fps)):
                            ration= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[9]/td/div/div/table/tbody/tr['+str(n+2)+']/td[2]/a')
                            ration_text= ration.text
                            ration.click()
                            
                            '/html/body/form/table'
                            rcd_table = main_browser.find_element_by_xpath('/html/body/form/table')
                            html = '<table>' + rcd_table.get_attribute('innerHTML') + '</table>'
                            html = BS(html)
                            try:
                                table2_xpath= main_browser.find_element_by_xpath('/html/body/form/table/tbody/tr[11]/td/div[2]/div/table')
                                df2= tableToDf(main_browser, table2_xpath)
                                df2.to_csv('./data/DistrictWiseCategoryDetails/rural/block/panch/village/fps/ration/table2_'+name+'.csv', index= False, encoding= 'utf-8'')
                            except:
                                pass
                            time.sleep(2)
                            break
                        break
                    break
                break
            break
        break
    return html

In [64]:
main_browser = chromeDriver()
html= fps(main_browser)

<ipython-input-11-8d000f15751c>:25: DeprecationWarning: use options instead of chrome_options
  main_browser = webdriver.Chrome(executable_path = path_to_chromedriver, chrome_options = options)


In [16]:
def tableToDf(browser, table_xpath):
    table = browser.find_element_by_xpath(table_xpath)
    html = '<table>' + table.get_attribute('innerHTML') + '</table>'
    html = BS(html)
    try:
        header = [[th.text for th in tr.findAll('th')] for tr in html.findAll('tr')]
    except:
        pass
    rows = [[td.text for td in tr.findAll('td')] for tr in html.findAll('tr')]
    df = pd.DataFrame(rows)
    if header:
        df.columns = header[0]
    df.drop(0, inplace = True)
    return df

In [7]:
def fps(main_browser):
    
    main_browser.get('https://jansoochna.rajasthan.gov.in/index.php/pds-fps-2/')

    for i in range(1,3):
        select_type= main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[2]/input['+str(i)+']')
        select_type.click()
        type_text= select_type.text
        time.sleep(0.5)

        select_dists = main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[1]/select')
        select_dists.click()
        dists = select_area_type.find_elements_by_tag_name("option")

        for d in range(len(dists)):
            district= main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[1]/select/option['+str(d+2)+']')
            district_text = district.text
            district.click()
            time.sleep(0.5)
            
            select_block = main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div[1]/select')
            select_block.click()
            blocks = select_area_type.find_elements_by_tag_name("option")

            for b in range(len(blocks)):
                block= main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div[1]/select/option['+str(b+2)+']')
                block.click()
                block_text = block.text
                time.sleep(0.5)
                
                select_ward = main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div[2]/select')
                select_ward.click()
                wards = select_area_type.find_elements_by_tag_name("option")
                
                for w in range(len(wards)):
                    ward= main_browser.find_element_by_xpath('')
                    
                    search= main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[1]/div/div/div/div/div/section[2]/div/div/div/div/div/div/div/div/div[2]/div/div[4]/div[1]/i')
                    search.click()
                    
                    table_main= main_browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/div/section[2]/div/div/div/div/div/section/div/div/div/div/div/div[1]/div/div[1]/table')
                    html = '<table>' + table.get_attribute('innerHTML') + '</table>'
                    html = BS(html)
                    header = [[th.text for th in tr.findAll('th')] for tr in html.findAll('tr')]
                    rows = [[td.text for td in tr.findAll('td')] for tr in html.findAll('tr')]
                    df = pd.DataFrame(rows)
                    df.columns = header[0]
                    df.loc[:,'area_type'] = area_type_text
                    df.loc[:,'district'] = district_text
                    df.drop(0, inplace = True)
                break
            break
        break

In [3]:
 for a in range(1,2):
            
        area_type = main_browser.find_element_by_xpath('/html/body/form/div[3]/div[3]/div[1]/table/tbody/tr[3]/td[2]/select/option['+str(a+2)+']')
        area_type.click()
        area_type_text = area_type.text.strip('--')
        time.sleep(0.5)
        select_district = main_browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlDistrict"]')
        select_district.click()
        districts = select_district.find_elements_by_tag_name("option")

        for d in range(len(districts)):

            district = main_browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlDistrict"]/option['+str(d+2)+']')
            district_text = district.text
            district.click()
            time.sleep(0.5)
            select_block = main_browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlBlockNagar"]')
            select_block.click()
            blocks = select_block.find_elements_by_tag_name("option")

            for b in range(len(blocks)):
                
                try:
                
                    block = main_browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlBlockNagar"]/option['+str(b+2)+']')
                    block.click()
                    block_text = block.text
                    
                    if os.path.exists('../Data/FairPriceShop/'+area_type_text+'/'+district_text+'/'+block_text+'.csv'):
                        continue
                        
                    time.sleep(1)
                    table = main_browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_GridViewFPSRuralUrban"]')
                    html = '<table>' + table.get_attribute('innerHTML') + '</table>'
                    html = BS(html)
                    header = [[th.text for th in tr.findAll('th')] for tr in html.findAll('tr')]
                    rows = [[td.text for td in tr.findAll('td')] for tr in html.findAll('tr')]
                    df = pd.DataFrame(rows)
                    df.columns = header[0]
                    df.loc[:,'area_type'] = area_type_text
                    df.loc[:,'district'] = district_text
                    df.drop(0, inplace = True)

                    print(f"are_type: {area_type_text}\tdistrict: {district_text}\tblock: {block_text}")

                    if not os.path.isdir('../Data/FairPriceShop/'+area_type_text+'/'+district_text+'/'):
                        os.makedirs('../Data/FairPriceShop/'+area_type_text+'/'+district_text)

                    df.to_csv('../Data/FairPriceShop/'+area_type_text+'/'+district_text+'/'+block_text+'.csv',index = False,encoding = 'utf-8')

                except Exception as e:
            
                    print(f"Exception {e} in are_type: {area_type_text}\tdistrict: {district_text}\tblock: {block_text}")
                    time.sleep(2)
                    continue

In [1]:
main_browser = chromeDriver()
fps(main_browser)